In [35]:
path_train = "data/painter-by-numbers/train/"
path_valid = "data/painter-by-numbers/valid/"

In [3]:
path_info = "data/painter-by-numbers/train_info.csv"

In [15]:
import os, json
from glob import glob
import numpy as np

In [12]:
g = glob(path_train+"*.jpg")

In [14]:
len(g)

79433

In [16]:
np.random.shuffle(g)

In [18]:
valid_files=g[:14800]

In [26]:
for f in valid_files:
    nf = f.replace("/train","/valid")
    os.rename(f,nf)
    

In [36]:
#base_path=path_train
base_path=path_valid
import csv
with open(path_info, 'rb') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    next(spamreader)
    for row in spamreader:
        artist = row[1]
        fileName = row[0]
        if not os.path.exists(base_path+artist):
            os.mkdir(base_path+artist)
        if os.path.exists(base_path+fileName):
            os.rename(base_path+fileName,base_path+artist+"/"+fileName)
    #print len(artists)